# Modélisation de l’incertitude — Master 2 MIAGE IA2
## TD 3 : Réseaux bayésiens
## Devoir de : MOSBAH Yasmin

# Installation de Cython et de Pomegranate

In [1]:
!pip3 install Cython
!pip3 install pomegranate
#Nous allons commencer par importer du module Pomegranate les éléments dont nous aurons besoin :
from pomegranate import *

# **Licencié ou augmenté ?**

In [2]:
#Nous créons le réseau :
G = BayesianNetwork('Bayes&Co')

#Créons maintenant les variables aléatoires de notre réseau :

#Le sujet nous dit qu'on a autant de chance de se faire licencier qu'auguementer 
licenciement = DiscreteDistribution({'Oui': 0.5, 'Non': 0.5})

#Ensuite le sujet nous présente chaque probabilité par membre des ressources humaines
rh = ConditionalProbabilityTable(
  [['Oui','Janet', 0.30],
   ['Non','Janet', 0.20],
   ['Oui','Jim', 0.35],
   ['Non','Jim', 0.20],
   ['Oui','Julia', 0.35],
   ['Non','Julia', 0.60]], [licenciement])

In [3]:
#Ensuite on passe dans la partie comptabilité
comptabilite = ConditionalProbabilityTable(
  [['Oui','James', 0.37],
   ['Non','James', 0.42],
   ['Oui','Jules', 0.63],
   ['Non','Jules', 0.58]], [licenciement])

In [4]:
#On ajoute la probabilité qu'il recoive un mail ou un courrier
notice = ConditionalProbabilityTable(
  [['Janet','James','Lettre', 0.40],
   ['Janet','James','Mail', 0.60],
   ['Janet','Jules','Lettre', 0.70],
   ['Janet','Jules','Mail', 0.30],
   ['Jim','James','Lettre', 0.30],
   ['Jim','James','Mail', 0.70],
   ['Jim','Jules','Lettre', 0.65],
   ['Jim','Jules','Mail', 0.35],
   ['Julia','James','Lettre', 0.25],
   ['Julia','James','Mail', 0.75],
   ['Julia','Jules','Lettre', 0.55],
   ['Julia','Jules','Mail', 0.45]], [rh,comptabilite])

In [5]:
# Ensuite, il faut créer la structure du réseau:
l = State(licenciement, name = 'licenciement')
r = State(rh, name = 'rh')
c = State(comptabilite, name = 'comptabilite')
n = State(notice, name = 'notice')
G.add_states(l,r,c,n)
G.add_edge(l,c)
G.add_edge(l,r)
G.add_edge(r,n)
G.add_edge(c,n)
G.bake()
#print(G)

## **Q1 :** Jeremy a reçu ce matin une lettre de Janet et Jules ; quelle est la probabilité qu’il s’agisse d’une lettre de licenciement ?

In [6]:
beliefs = G.predict_proba({'notice': 'lettre','rh': 'Janet','comptabilite': 'Jules'})
beliefs = map(str, beliefs)
for state, belief in zip(G.states, beliefs):
    print(state.name, belief)

licenciement {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "Oui" : 0.619672131147541,
            "Non" : 0.380327868852459
        }
    ],
    "frozen" : false
}
rh Janet
comptabilite Jules
notice lettre


## **RESULTAT :** Le resultat nous indique que Jeremy qui a reçu une lettre de Janet et de Jules a 62% de chances que ca soit un licenciement 

__________________________________________________________________________
## **Q2 :** Et s’il a reçu une lettre de Jim et James ?

In [7]:
beliefs = G.predict_proba({'notice': 'lettre','rh': 'Jim','comptabilite': 'James'})
beliefs = map(str, beliefs)
for state, belief in zip(G.states, beliefs):
    print(state.name, belief)

licenciement {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "Oui" : 0.6065573770491803,
            "Non" : 0.3934426229508198
        }
    ],
    "frozen" : false
}
rh Jim
comptabilite James
notice lettre


## **RESULTAT :** Le resultat nous indique que si Jeremy avait reçu une lettre de JIm et James, il aurait 60.5 % de chances de recevoir une lettre de licenciment. 

_________________________________________________________________________
## **Q3 :**  Si tout ce que Jeremy sait, c’est que Julia est en charge d’un dossier le concernant, quels sont ses risques d’être licencié ?

In [8]:
beliefs = G.predict_proba({'rh': 'Julia'})
beliefs = map(str, beliefs)
for state, belief in zip(G.states, beliefs):
    print(state.name, belief)

licenciement {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "Oui" : 0.3684210526315789,
            "Non" : 0.6315789473684211
        }
    ],
    "frozen" : false
}
rh Julia
comptabilite {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "James" : 0.4015789473684212,
            "Jules" : 0.5984210526315789
        }
    ],
    "frozen" : false
}
notice {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "Mail" : 0.5704736842105264,
            "Lettre" : 0.42952631578947353
        }
    ],
    "frozen" : false
}


## **RESULTAT :** Si il sait suelement que c'est Julia qui ets en charge de son dossier, alors il a seulement 37% de chances d'etre licencié

___________________________________________________________________________
## **Q4 :**  Jeremy a reçu un mail ce matin, mais il ne sait pas qui a traité son dossier. Quelle probabilité a-t-il d’y lire une annonce d’augmentation de salaire ?

In [9]:
beliefs = G.predict_proba({'notice': 'Mail'})
beliefs = map(str, beliefs)
for state, belief in zip(G.states, beliefs):
    print(state.name, belief)

licenciement {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "Oui" : 0.4790334104431272,
            "Non" : 0.5209665895568728
        }
    ],
    "frozen" : false
}
rh {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "Jim" : 0.2626267431628098,
            "Julia" : 0.5324988272300533,
            "Janet" : 0.20487442960713706
        }
    ],
    "frozen" : false
}
comptabilite {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "James" : 0.5429269144877167,
            "Jules" : 0.45707308551228326
        }
    ],
    "frozen" : false
}
notice Mail


RESULTAT : Il a 52% de chances d'avoir une annonce pour une augumentation de salaire.

# **Enquête**

In [10]:
from pomegranate import *

#Nous créons le réseau :
G = BayesianNetwork('BNTech')
#On nous dit que le coupable a agit seul et il y a au total 100 employés
coupable = DiscreteDistribution({'Oui': 0.01, 'Non': 0.99})

#On nous dit que le coupable a 45% de chance de porter des gants (donc 55% de pas en porter)
#On nous indique également que la probabilité de mettre des gants sans faire de cambriolage
# peut être considérée comme nulle 
gant = ConditionalProbabilityTable(
  [['Oui','Oui', 0.45],
   ['Oui','Non', 0.55],
   ['Non','Oui', 0.001],
   ['Non','Non', 0.999]],
   [coupable])

'''
empreinte = ConditionalProbabilityTable(
  [['Oui','Oui', 0.02],
   ['Oui','Non', 0.99],
   ['Non','Oui', 0.98],
   ['Non','Non', 0.01]],
   [gant])

empreinte = ConditionalProbabilityTable(
  [['Oui','Oui', 0.95],
   ['Oui','Non', 0],
   ['Non','Oui', 0.05],
   ['Non','Non', 1]],
   [coupable])

'''

empreinte = ConditionalProbabilityTable(
  [['Oui','Oui','Oui', 0.02],
   ['Oui','Oui','Non',0.98 ],
   
   ['Oui','Non','Oui', 0.001], #
   ['Oui','Non','Non', 0.999], #
   
   ['Non','Oui','Oui', 0.99],
   ['Non','Oui','Non', 0.01],
   
   ['Non','Non','Oui', 0.05],
   ['Non','Non','Non', 0.95]],
   [gant,coupable])


# On nous dit que les tests ADN ont 0.1% de faux négatifs et de faux positifs
adn =  ConditionalProbabilityTable(
  [['Oui','Oui', 0.999],
   ['Oui','Non', 0.001],
   ['Non','Oui', 0.001],
   ['Non','Non', 0.999]],
   [coupable])

In [11]:
# Ensuite, il faut créer la structure du réseau:
c = State(coupable, name = 'coupable')
g = State(gant, name = 'gant')
e = State(empreinte, name = 'empreinte')
a = State(adn, name = 'adn')

G.add_states(c,g,e,a)
G.add_edge(c,g)
G.add_edge(g,e)
G.add_edge(c,e)
G.add_edge(c,a)
G.bake()

In [12]:
#En ce point, on n'a seulement les empreintes de Koul ne correspondent pas a celles trouvées 
# sur les lieux du cambriolage.
beliefs = G.predict_proba({'empreinte': 'Non'})
beliefs = map(str, beliefs)
for state, belief in zip(G.states, beliefs):
    print(state.name, belief)

coupable {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "Oui" : 0.000120432667393224,
            "Non" : 0.9998795673326069
        }
    ],
    "frozen" : false
}
gant {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "Non" : 0.9988820227552336,
            "Oui" : 0.0011179772447664452
        }
    ],
    "frozen" : false
}
empreinte Non
adn {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "Non" : 0.9988798081979415,
            "Oui" : 0.0011201918020584604
        }
    ],
    "frozen" : false
}


## La probabilité de culpabilité de Pierre Koul au point 1 est de moins de 0.05%
--------------------------------------------------------------------------------------------

In [13]:
#En ce point on nous indique que ses cheveux correspondent a l'adn trouvé sur les lieux du crime
beliefs = G.predict_proba({'empreinte':'Non','adn':'Oui'})
beliefs = map(str, beliefs)
for state, belief in zip(G.states, beliefs):
    print(state.name, belief)

coupable {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "Oui" : 0.9916997031989,
            "Non" : 0.008300296801100013
        }
    ],
    "frozen" : false
}
gant {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "Non" : 0.023374300263427635,
            "Oui" : 0.9766256997365723
        }
    ],
    "frozen" : false
}
empreinte Non
adn Oui


## La probabilité de culpabilité de Pierre Koul au point 3 est de 0.99%
--------------------------------------------------------------------------------------------

In [14]:
#En ce point, l’hypothèse que Pierre portait des gants est écartée
beliefs = G.predict_proba({"empreinte":"Non",'adn':'Oui',"gant":"Non"})
beliefs = map(str, beliefs)
for state, belief in zip(G.states, beliefs):
    print(state.name, belief)

coupable {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "Oui" : 0.05524861878472231,
            "Non" : 0.9447513812152777
        }
    ],
    "frozen" : false
}
gant Non
empreinte Non
adn Oui


## La probabilité de culpabilité de Pierre Koul au point 3 est de 5.5%
--------------------------------------------------------------------------------------------